# Getting the raw dataset

We'll be using `twython` and `cryptocompare` as APIs to get the respective
Twitter and cryptocurrency data that will make up the project's dataset.

## Importing packages.

First, we'll import the necessary python packages.

In [1]:
# Access environment variables.
from os import environ

# Cryptocompare API
import cryptocompare

# Manipulating the raw data to save it in a ``.csv`` files.
import pandas as pd

# Loading environment variables from a `.env` file.
from dotenv import load_dotenv

# Twython API.
from twython import Twython

## Loading secrets from environment variables.

Next, we need to set credentials to access the CryptoCompare and Twython
APIs. To avoid hard coding the secrets, we'll load them from the environment,
i.e., a `.env` file.

In [2]:
# Loads environment variables from a `.env` file.
load_dotenv()

# Now the environment variables from the file are available, as if they were
# specified typically from the commandline.
TWITTER_APP_KEY = environ["TWITTER_APP_KEY"]
TWITTER_APP_SECRET = environ["TWITTER_APP_SECRET"]

## Testing the APIs.

Let's test the CryptoCompare API. `CRYPTOCOMPARE_API_KEY` should be specified
in the `.env` file so that the python package can detect it automatically as
an environment variable.

In [3]:
cryptocompare.get_price("BTC", currency="USD")

{'BTC': {'USD': 40802.6}}

Now let's test accessing Twitter's API through Twython.

In [4]:
twitter = Twython(TWITTER_APP_KEY, TWITTER_APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(
    TWITTER_APP_KEY, TWITTER_APP_SECRET, access_token=ACCESS_TOKEN
)

search_results = twitter.search(count=1, q="cryptocurrency")
print(search_results)

{'statuses': [{'created_at': 'Fri Mar 04 18:29:36 +0000 2022', 'id': 1499814332003414016, 'id_str': '1499814332003414016', 'text': 'RT @CryptoTownEU: 🚀 Airdrop: AvaStation\n💰 Value: 250 $AVS\n👥 Top 1000 Referral: 250 $AVS\n👥 Top 50 Referral: 20 $BUSD\n📅 End Date: 25th March,…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [{'text': 'AVS', 'indices': [53, 57]}, {'text': 'AVS', 'indices': [83, 87]}, {'text': 'BUSD', 'indices': [110, 115]}], 'user_mentions': [{'screen_name': 'CryptoTownEU', 'name': 'CryptoTown', 'id': 1034478294677250048, 'id_str': '1034478294677250048', 'indices': [3, 16]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1434172225561579526, 'id_str': '1434172225561579